In [1]:
import pandas as pd
import glob

import os
from skimage import io, measure, morphology, color, filters, transform
import numpy as np

from matplotlib import pyplot as plt
import cv2
from tqdm.notebook import tqdm


import porespy as ps

from statsmodels.tsa.api import SimpleExpSmoothing
from scipy.signal import find_peaks
import more_itertools as mit

import urllib
# import torch
import numba

C:\anaconda3\envs\torch\lib\site-packages\openpnm\algorithms\_invasion_percolation.py:358: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _find_trapped_pores(inv_seq, indices, indptr, outlets):  # pragma: no cover


## Split text by unique word's contours (unsuccessful)

In [ ]:
def rgb2smyk(bgr):
    bgrdash = bgr.astype(np.float32)/255.

    # Calculate K as (1 - whatever is biggest out of Rdash, Gdash, Bdash)
    K = 1 - np.max(bgrdash, axis=2)

    # Calculate C
    C = (1-bgrdash[...,2] - K)/(1-K)

    # Calculate M
    M = (1-bgrdash[...,1] - K)/(1-K)

    # Calculate Y
    Y = (1-bgrdash[...,0] - K)/(1-K)

    K*=255
    K=K.astype(np.uint8)

    C*=255
    C=C.astype(np.uint8)

    M*=255
    M=M.astype(np.uint8)

    Y*=255
    Y=Y.astype(np.uint8)

    return C,M,Y,K

## Hausford (fractal) dimention (done)

In [13]:
# ровный
name='00201_73.jpg'

# вправо сильно
# name='67205_3.jpg'

#  ровный
# name='62325_1_12.jpg'

# наклон вправо
# name='67205_2.jpg'


# name='1200px-Sierpinski_triangle.svg.png'

img=io.imread(name)

(C, M, Y, K) = rgb2smyk(img)

thresh=filters.threshold_otsu(Y)
binaryImage=Y>thresh

# input binary gray
data = ps.metrics.boxcount(binaryImage)

print(data.slope)
print(data.size)

  0%|          | 0/10 [00:00<?, ?it/s]

[1.3376751  1.32116039 1.51325804 1.81841289 1.89520178 1.90472269
 1.68188838 1.92649966 1.46718892 0.50593317]
[  10   17   31   55   97  172  303  536  948 1674]


In [ ]:
df=pd.read_csv('grades.csv')

recs={'msk22':[[780,2610],[36, 1838]],
      'msk23':[[763,2621],[44, 1824]],
      'nov23':[[755,2590],[41, 1834]],}

fractal_dim=[]

for i in tqdm(range(len(df))):
    id=df.iloc[i]['id']
    type=df.iloc[i]['type']
    rec=recs[type]
    img=io.imread(f'data/{type}/blanks_aligned_jpg/{id}.jpg')[rec[0][0]:rec[0][1],rec[1][0]:rec[1][1]]

    (C, M, Y, K) = rgb2smyk(img)

    thresh=filters.threshold_otsu(Y)
    binaryImage=Y>thresh*0.7
    data = ps.metrics.boxcount(binaryImage)

    fractal_dim.append(data.slope[0])
    
    if i%10==0:
        np.save('fractal_dim.npy',fractal_dim)


  0%|          | 0/21308 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [5]:
df=pd.read_csv('grades.csv',nrows=21301)
df['fractal_dim']=np.load('fractal_dim.npy')

In [3]:
len(df)

21308

In [4]:
np.load('fractal_dim.npy').shape

(21301,)

In [6]:
df.to_csv('fracral_grades.csv', index=False)